# Using the Equities India (NSE) data-set

In [1]:
from sqlalchemy import func, and_, or_
from plutoPy.model import RoutingSession, EquitiesIndiaNse
from datetime import date, datetime

### fetch the earliest 10 listed equity

In [2]:
results = (RoutingSession.session.query(EquitiesIndiaNse.Tickers)
           .order_by(EquitiesIndiaNse.Tickers.DATE_LISTING)
           .limit(10)
           .all())

for instance in results:
    print(instance)

AARTIIND(BE): Aarti Industries Limited, 1995-Feb-08
ABB(EQ): ABB India Limited, 1995-Feb-08
AMJLAND(EQ): Amj Land Holdings Limited, 1995-Feb-08
ARVIND(EQ): Arvind Limited, 1995-Feb-08
BALRAMCHIN(EQ): Balrampur Chini Mills Limited, 1995-Feb-08
BLUEBLENDS(BE): Blue Blends (I) Limited, 1995-Feb-08
CESC(EQ): CESC Limited, 1995-Feb-08
CIPLA(EQ): Cipla Limited, 1995-Feb-08
EIHOTEL(EQ): EIH Limited, 1995-Feb-08
FEDERALBNK(EQ): The Federal Bank  Limited, 1995-Feb-08


/usr/local/lib/python3.6/dist-packages/sqlalchemy/dialects/mssql/base.py:2307: SAWarning: Could not fetch transaction isolation level, tried views: ('sys.dm_exec_sessions', 'sys.dm_pdw_nodes_exec_sessions'); final error was: ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'sys.dm_pdw_nodes_exec_sessions'.")
  "tried views: %s; final error was: %s" % (views, last_error)


### fetch some "misc" info for State Bank of India   

In [3]:
end_dt = RoutingSession.session.query(func.max(EquitiesIndiaNse.MiscInfo.TIME_STAMP)).scalar() 

results = (RoutingSession.session.query(EquitiesIndiaNse.MiscInfo)
            .filter(and_(EquitiesIndiaNse.MiscInfo.TIME_STAMP == end_dt, EquitiesIndiaNse.MiscInfo.SYMBOL == 'SBIN'))
            .all())

print("misc info for SBIN:")
for instance in results:
    print(instance)

misc info for SBIN:
SBIN/2019-Jul-16: 139822.34,53.15,324.05,396.05


### fetch the market-cap decile of DHFL since we started capturing the data-set

In [4]:
results = (RoutingSession.session.query(EquitiesIndiaNse.MarketCapDecile)
            .filter(EquitiesIndiaNse.MarketCapDecile.SYMBOL == 'DHFL')
            .all())

print("market-cap deciles for DHFL over time:")
for instance in results:
    print(instance)

market-cap deciles for DHFL over time:
DHFL/2014-Aug-28: 1
DHFL/2014-Sep-25: 1
DHFL/2014-Oct-30: 1
DHFL/2014-Nov-27: 1
DHFL/2014-Dec-24: 1
DHFL/2015-Jan-29: 1
DHFL/2015-Feb-26: 1
DHFL/2015-Mar-26: 1
DHFL/2015-Apr-30: 1
DHFL/2015-May-28: 1
DHFL/2015-Jun-25: 1
DHFL/2015-Jul-30: 1
DHFL/2015-Aug-27: 1
DHFL/2015-Sep-24: 1
DHFL/2015-Oct-29: 1
DHFL/2015-Nov-26: 1
DHFL/2015-Dec-31: 1
DHFL/2016-Jan-28: 1
DHFL/2016-Feb-25: 1
DHFL/2016-Mar-31: 1
DHFL/2016-Apr-28: 1
DHFL/2016-May-26: 1
DHFL/2016-Jun-30: 1
DHFL/2016-Jul-28: 1
DHFL/2016-Aug-25: 1
DHFL/2016-Sep-29: 1
DHFL/2016-Oct-27: 1
DHFL/2016-Nov-24: 1
DHFL/2016-Dec-29: 1
DHFL/2017-Jan-25: 1
DHFL/2017-Feb-23: 1
DHFL/2017-Mar-30: 0
DHFL/2017-Apr-27: 0
DHFL/2017-May-25: 0
DHFL/2017-Jun-29: 0
DHFL/2017-Jul-27: 0
DHFL/2017-Aug-31: 0
DHFL/2017-Sep-28: 0
DHFL/2017-Oct-26: 0
DHFL/2017-Nov-30: 0
DHFL/2017-Dec-28: 0
DHFL/2018-Jan-25: 0
DHFL/2018-Feb-22: 0
DHFL/2018-Mar-28: 0
DHFL/2018-Apr-26: 0
DHFL/2018-Jun-28: 0
DHFL/2018-Jul-26: 0
DHFL/2018-Aug-30: 0
D

### fetch the latest end-of-day prices for State Bank of India

In [5]:
end_dt = RoutingSession.session.query(func.max(EquitiesIndiaNse.EodTimeSeries.TIME_STAMP)).scalar()

results = (RoutingSession.session.query(EquitiesIndiaNse.EodTimeSeries)
            .filter(and_(EquitiesIndiaNse.EodTimeSeries.TIME_STAMP == end_dt, EquitiesIndiaNse.EodTimeSeries.SYMBOL == 'SBIN'))
            .all())

print(f"fetched: {len(results)}")
for instance in results:
    print(instance)

fetched: 4
SBIN(EQ)/2019-Jul-15: 365.0,357.6,364.1,360.05,15665939.0
SBIN(N2)/2019-Jul-15: 11250.05,11250.0,11250.0,11250.05,80.0
SBIN(N5)/2019-Jul-15: 11470.0,11375.5,11450.0,11441.15,191.0
SBIN(N6)/2019-Jul-15: 11250.0,11200.0,11249.75,11200.0,11.0


### fetch the last 10 day EOD prices for State Bank of India

In [6]:
results = (RoutingSession.session.query(EquitiesIndiaNse.EodTimeSeries)
            .filter(and_(or_(EquitiesIndiaNse.EodTimeSeries.SERIES == 'EQ', EquitiesIndiaNse.EodTimeSeries.SERIES == 'BE'), 
                         EquitiesIndiaNse.EodTimeSeries.SYMBOL == 'SBIN'))
            .order_by(EquitiesIndiaNse.EodTimeSeries.TIME_STAMP.desc())
            .limit(10)
            .all())

for instance in results:
    print(instance)   

SBIN(EQ)/2019-Jul-15: 365.0,357.6,364.1,360.05,15665939.0
SBIN(EQ)/2019-Jul-12: 366.55,361.5,363.0,363.6,14661748.0
SBIN(EQ)/2019-Jul-11: 364.0,357.0,358.5,363.2,20048321.0
SBIN(EQ)/2019-Jul-10: 361.5,351.3,360.55,354.2,18681819.0
SBIN(EQ)/2019-Jul-09: 360.0,352.5,353.5,359.5,16579914.0
SBIN(EQ)/2019-Jul-08: 368.45,353.5,368.0,355.3,27247892.0
SBIN(EQ)/2019-Jul-05: 373.6,366.0,369.45,370.65,30019347.0
SBIN(EQ)/2019-Jul-04: 370.9,366.2,367.25,367.4,19252646.0
SBIN(EQ)/2019-Jul-03: 366.7,363.0,365.25,366.15,12626285.0
SBIN(EQ)/2019-Jul-02: 365.0,360.35,362.15,364.5,14482172.0


### UPL did a 1:2 bonus on 2019-07-02. see unadjusted eod vs. adjusted eod

In [7]:
startDt = datetime(2019, 6, 15)    
endDt = datetime(2019, 7, 15)

print("unadjusted eod")
results = (RoutingSession.session.query(EquitiesIndiaNse.EodTimeSeries)
            .filter(and_(or_(EquitiesIndiaNse.EodTimeSeries.SERIES == 'EQ', EquitiesIndiaNse.EodTimeSeries.SERIES == 'BE'), 
                         EquitiesIndiaNse.EodTimeSeries.SYMBOL == 'UPL',
                         EquitiesIndiaNse.EodTimeSeries.TIME_STAMP >= startDt,
                         EquitiesIndiaNse.EodTimeSeries.TIME_STAMP <= endDt))
            .order_by(EquitiesIndiaNse.EodTimeSeries.TIME_STAMP)
            .all())

for instance in results:
    print(instance)

print("adjusted eod")    
results = (RoutingSession.session.query(EquitiesIndiaNse.EodAdjustedTimeSeries)
            .filter(and_(EquitiesIndiaNse.EodAdjustedTimeSeries.SYMBOL == 'UPL',
                         EquitiesIndiaNse.EodAdjustedTimeSeries.TIME_STAMP >= startDt,
                         EquitiesIndiaNse.EodAdjustedTimeSeries.TIME_STAMP <= endDt))
            .order_by(EquitiesIndiaNse.EodAdjustedTimeSeries.TIME_STAMP)
            .all())

for instance in results:
    print(instance)  

unadjusted eod
UPL(EQ)/2019-Jun-17: 1013.8,991.0,1007.5,994.95,1437711.0
UPL(EQ)/2019-Jun-18: 1003.5,988.0,994.0,999.7,1125228.0
UPL(EQ)/2019-Jun-19: 1005.85,936.9,1003.0,950.3,2657234.0
UPL(EQ)/2019-Jun-20: 940.55,833.0,940.55,870.25,12905733.0
UPL(EQ)/2019-Jun-21: 914.75,868.15,876.0,890.1,12447297.0
UPL(EQ)/2019-Jun-24: 941.6,895.2,903.4,936.55,6506356.0
UPL(EQ)/2019-Jun-25: 939.8,910.45,933.9,934.95,5140843.0
UPL(EQ)/2019-Jun-26: 963.8,922.1,931.0,958.45,3365399.0
UPL(EQ)/2019-Jun-27: 961.8,937.15,952.15,943.5,5241680.0
UPL(EQ)/2019-Jun-28: 951.0,918.65,946.0,937.45,3292847.0
UPL(EQ)/2019-Jul-01: 957.0,935.55,945.0,948.2,2692425.0
UPL(EQ)/2019-Jul-02: 658.9,625.05,637.4,656.15,6615515.0
UPL(EQ)/2019-Jul-03: 662.65,648.1,658.5,651.8,2997163.0
UPL(EQ)/2019-Jul-04: 705.95,652.9,654.85,698.35,12488054.0
UPL(EQ)/2019-Jul-05: 709.05,660.8,706.9,665.8,12058104.0
UPL(EQ)/2019-Jul-08: 673.5,653.0,669.95,662.1,4403598.0
UPL(EQ)/2019-Jul-09: 660.0,632.25,660.0,640.05,7434335.0
UPL(EQ)/2019-Ju

### fetch the last 10 day returns for State Bank of India

In [8]:
results = (RoutingSession.session.query(EquitiesIndiaNse.DailyReturns)
            .filter(EquitiesIndiaNse.DailyReturns.SYMBOL == 'SBIN')
            .order_by(EquitiesIndiaNse.DailyReturns.TIME_STAMP.desc())
            .limit(10)
            .all())

for instance in results:
    print(instance)       

SBIN/2019-Jul-15: -0.009763
SBIN/2019-Jul-12: 0.001101
SBIN/2019-Jul-11: 0.025409
SBIN/2019-Jul-10: -0.014743
SBIN/2019-Jul-09: 0.011821
SBIN/2019-Jul-08: -0.041414
SBIN/2019-Jul-05: 0.008846
SBIN/2019-Jul-04: 0.003414
SBIN/2019-Jul-03: 0.004527
SBIN/2019-Jul-02: 0.008159


### fetch the last 10 corporate actions for State Bank of India

In [9]:
results = (RoutingSession.session.query(EquitiesIndiaNse.CorporateActions)
            .filter(EquitiesIndiaNse.CorporateActions.SYMBOL == 'SBIN')
            .order_by(EquitiesIndiaNse.CorporateActions.EX_DATE.desc())
            .limit(10)
            .all())

for instance in results:
    print(instance)

SBIN(EQ)/2018-Jun-15:  ANNUAL GENERAL MEETING/ CHANGE IN REGISTRAR AND TRANSFER AGENT
SBIN(EQ)/2017-May-26:  DIVIDEND
SBIN(EQ)/2016-Jun-03:  FINAL DIVIDEND
SBIN(EQ)/2015-May-28: DIVIDEND
SBIN(EQ)/2014-Nov-20: FACE VALUE SPLIT RS.10/- TO RE.1/- PER SHARE
SBIN(EQ)/2014-May-29: AGM/DIVIDEND


### fetch the last 24 quarter EPS for State Bank of India

In [10]:
refIds = (RoutingSession.session.query(EquitiesIndiaNse.CorporateResultsMeta)
            .filter(and_(EquitiesIndiaNse.CorporateResultsMeta.SYMBOL == 'SBIN'),
                    EquitiesIndiaNse.CorporateResultsMeta.IS_CONSOLIDATED == False,
                    EquitiesIndiaNse.CorporateResultsMeta.PERIOD.ilike('%quarter'))
            .order_by(EquitiesIndiaNse.CorporateResultsMeta.PERIOD_END.desc())
            .limit(24)
            .all())

for instance in refIds:
    print(instance)
    
    results = (RoutingSession.session.query(EquitiesIndiaNse.CorporateResults)
               .filter(and_(EquitiesIndiaNse.CorporateResults.REF_ID == instance.REF_ID,
                            EquitiesIndiaNse.CorporateResults.KEY.ilike('%diluted%before%')))
               .all())
    
    for r in results:
        print(r)

SBIN(True/False/False) [Fourth Quarter: 2019-Jan-01 ~ 2019-Mar-31]
None/None/None/None/Diluted EPS before Extraordinary items (in Rs.): 0.94
SBIN(False/False/False) [Third Quarter: 2018-Oct-01 ~ 2018-Dec-31]
None/None/None/None/Diluted EPS before Extraordinary items (in Rs.): 4.43
SBIN(False/False/False) [Second Quarter: 2018-Jul-01 ~ 2018-Sep-30]
None/None/None/None/Diluted EPS before Extraordinary items (in Rs.): 1.06
SBIN(False/False/False) [First Quarter: 2018-Apr-01 ~ 2018-Jun-30]
None/None/None/None/Diluted EPS before Extraordinary items (in Rs.): -5.46
SBIN(True/False/False) [Fourth Quarter: 2018-Jan-01 ~ 2018-Mar-31]
None/None/None/None/Diluted EPS before Extraordinary items (in Rs.): -8.92
SBIN(False/False/False) [Third Quarter: 2017-Oct-01 ~ 2017-Dec-31]
None/None/None/None/Diluted EPS before Extraordinary items (in Rs.): -2.8
SBIN(False/False/False) [Second Quarter: 2017-Jul-01 ~ 2017-Sep-30]
None/None/None/None/Diluted EPS before Extraordinary items (in Rs.): 1.83
SBIN(Fals

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)